In [1]:
import sqlalchemy
from os import environ

import numpy as np
import pandas as pd

from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## The next step is to model the Fantasy scores. A simple model would be calculating the expected scores for each player.

In [2]:
value = {"PTS":1,"BLK":4,"AST":2,"REB":1,"TOV":-2,"FGM":2,"FGA":-1,"FTM":1,"FTA":-1,"PM3":1,"STL":4}

### The fantasy scores are given as :
$$score = \sum^{N-1}_{i=0} v_i \cdot s_i$$

Where $v_i$ is the value assigned to stat $s_i$.

In [3]:
v = np.array(list(value.values()))

In [4]:
stats = ", ".join(list(value.keys()))

In [5]:
engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"+environ.get("USER")+\
                                  ":"+environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [6]:
fields = "Name, " + stats + ", Game_day "

join =  "Box_scores INNER JOIN Players on Box_scores.Player_ID = Players.ID "

select = "SELECT "+ fields + " FROM " + join + "ORDER BY Game_day desc"

In [7]:
box_scores = pd.read_sql(select,engine)

In [8]:
box_scores["Game_day"] = pd.to_datetime(box_scores["Game_day"])

## Calculate the fantasy points by a matrix multiplication
$$\vec{f} = \vec{v} \times \matrix{B} $$

Where $\vec{f}$ is the fantasy points, $\vec{v}$ is the values assigned to the box scores $\matrix{B}$.

In [9]:
def calc_fantasy_points(v, box_scores):
    return v @ box_scores

In [10]:
s = np.array(box_scores.values[:,1:-1],dtype = np.int64)

In [11]:
box_scores["Fantasy Points"] = calc_fantasy_points(v, s.T)

## The fantasy scores can then be grouped by the player names and aggregated using a rolling mean on their fantasy scores.

Typically, there are 3-4 games a week and since the Fantasy League is in a week-by week format, it makes sense to set the rolling mean window to 4. This will show how a player is doing on a week to week basis.

In [12]:
names = box_scores["Name"].unique()

In [17]:
rolls= box_scores[["Name","Fantasy Points"]].groupby("Name").mean()

In [21]:
names = box_scores["Name"].unique()
hist = pd.DataFrame(data = {}, index=names)

In [26]:
h = np.histogram(box_scores.loc[box_scores["Name"] == "Donovan Mitchell"]["Fantasy Points"]\
             .values,bins=250,range=(-50,200),density=True)

In [32]:
h

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02380952, 0.        , 0.        ,
        0.02380952, 0.        , 0.        , 0.  

In [53]:
len(h[0])

250

In [54]:
len(h[1])

251

In [116]:
prod  = np.multiply(h[0],h[1][:-1])

In [117]:
np.sum(prod)

42.47619047619047

In [77]:
sum(box_scores["Name"].isna())

0

In [106]:
def histogram(box_scores):
    names = box_scores["Name"].unique()
    hist = pd.DataFrame(columns=["Name","P(F)","F"])
    d = pd.DataFrame(columns=["Name","P(F)","F"])

    for n in names:
        player = box_scores.loc[box_scores['Name'].isin([n])]["Fantasy Points"].values
        h = np.histogram(player,bins=250,range=(-50,200),density=True)
        d["P(F)"] = h[0]
        d["F"] = h[1][:-1]
        d["Name"] = n
        hist = pd.concat((hist,d),ignore_index=True)
#         hist.loc[n,"F"] = h[1,np.newaxis]
        
    return hist

In [107]:
hist = histogram(box_scores)

In [109]:
def efp(hist):
    

,Name,P(F),F
0,Greg Monroe,0.0,-50.0
1,Greg Monroe,0.0,-49.0
2,Greg Monroe,0.0,-48.0
3,Greg Monroe,0.0,-47.0
4,Greg Monroe,0.0,-46.0
...,...,...,...
147745,Brook Lopez,0.0,195.0
147746,Brook Lopez,0.0,196.0
147747,Brook Lopez,0.0,197.0
147748,Brook Lopez,0.0,198.0


In [17]:
def options(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

## Below defines a dashboard that shows a line graph for aggregated fantasy scores against the game day. A searchable dropdown includes all the players who recorded stats this season. Simply search and select the player(s) of interest and their fantasy points over the course of the season will be visible. The dropdown includes functionality for multiple players to enable multiple comparisons. 

In [25]:
def drpdwn():
    opts = options(names)
    d = dcc.Dropdown(options = opts, value = names[0:2],id = 'dp',multi = True)
    
    return d

In [26]:
fig = px.line(y="Rolling", x="Game_day",color="Name",data_frame = box_scores.head(5), markers=True,range_y=(0,100))
fig.update_layout(title ="Fantasy points during current season", xaxis_title = "Game Day",
                  title_x = 0.5,yaxis_title = "4 game Rolling mean / Fantasy points")


app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#333333','radio_button':'#BBBBBB'} 
text_size = {'H1':48,'H2':40,'text':28,'radio_button':20}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'Arial'}, children=[
    html.H1(children="ESPN Fantasy Analysis",
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),
    
    html.Div(children=[drpdwn(), dcc.Graph(figure = fig, id = 'graph')])



])

@app.callback(
    Output('graph','figure'),
    Input('dp','value'))
def update_figure(selected):
    d = box_scores.loc[box_scores['Name'].isin(selected)]
    
    fig = px.line(y="Rolling", x="Game_day",color="Name",data_frame = d, markers=True,range_y=(0,100))
    
    fig.update_layout(title ="Fantasy points during current season", xaxis_title = "Game Day",
                  title_x = 0.5,yaxis_title = "4 game Rolling mean / Fantasy points")
    return fig

In [27]:
app.run_server(mode = "external")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/
